In [14]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

# Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 500 if 'CI' in os.environ else 2591

from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

session = SnorkelSession()
dev_cands = session.query(Spouse).filter(Spouse.split == 1).all()
test_cands = session.query(Spouse).filter(Spouse.split == 2).all()

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
# gold_labels_dev = [L[0,0] for L in L_gold_dev]
# gold_labels_test = [L[0,0] for L in L_gold_test]

from snorkel.learning.utils import MentionScorer

In [40]:
#gold_labels_dev = [x[0,0] for x in L_gold_dev.todense()]
#for i,L in enumerate(gold_labels_dev):
#    print(i,gold_labels_dev[i])

# gold_labels_dev = []
# for i,L in enumerate(L_gold_dev):
#     gold_labels_dev.append(L[0,0])
    
# gold_labels_test = []
# for i,L in enumerate(L_gold_test):
#     gold_labels_test.append(L[0,0])
    

print(gold_labels_dev.count(1),gold_labels_dev.count(-1))
print(len(gold_labels_dev))


189 2625
2814


In [17]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [18]:
##### Continuous ################

softmax_Threshold = 0.3
LF_Threshold = 0.3

import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)


spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for sw in spouses:
        sc=max(sc,get_similarity(word_vectors,sw))
    return (1,sc)

def LF_husband_wife_left_window(c):
    global LF_Threshold
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for sw in spouses:
        sc_1=max(sc_1,get_similarity(word_vectors,sw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for sw in spouses:
        sc_2=max(sc_2,get_similarity(word_vectors,sw))
    return(1,max(sc_1,sc_2))
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,0.75) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    global LF_Threshold
    word_vectors = get_word_vectors(preprocess(get_right_tokens(c)))
    sc = get_similarity(word_vectors,'married')
    
    if 'and' in get_between_tokens(c):
        return (1,sc)
    else:
        return (0,0)

def LF_familial_relationship(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for fw in family:
        sc=max(sc,get_similarity(word_vectors,fw))
        
    return (-1,sc) 

def LF_family_left_window(c):
    global LF_Threshold
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for fw in family:
        sc_1=max(sc_1,get_similarity(word_vectors,fw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for fw in family:
        sc_2=max(sc_2,get_similarity(word_vectors,fw))
        
    return (-1,max(sc_1,sc_2))

def LF_other_relationship(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for ow in other:
        sc=max(sc,get_similarity(word_vectors,ow))
        
    return (-1,sc) 

def LF_other_relationship_left_window(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c)))
    for ow in other:
        sc=max(sc,get_similarity(word_vectors,ow))
    return (-1,sc) 

import bz2

# Function to remove special characters from text
def strip_special(s):
    s = s.decode("utf-8") 
    return ''.join(c for c in s if ord(c) < 128)

# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
    known_spouses = set(
        tuple(strip_special(x).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,1)

import numpy as np

def LF_Three_Lists_Left_Window(c):
    global softmax_Threshold
    c1,s1 = LF_husband_wife_left_window(c)
    c2,s2 = LF_family_left_window(c)
    c3,s3 = LF_other_relationship_left_window(c)
    sc = np.array([s1,s2,s3])
    c = [c1,c2,c3]
    sharp_param = 1.5
    prob_sc = np.exp(sc * sharp_param - np.max(sc))
    prob_sc = prob_sc / np.sum(prob_sc)
    #print 'Left:',s1,s2,s3,prob_sc
    
    if s1==s2 or s3==s1:
        return (0,0)
    return c[np.argmax(prob_sc)],1

def LF_Three_Lists_Between_Words(c):
    global softmax_Threshold
    c1,s1 = LF_husband_wife(c)
    c2,s2 = LF_familial_relationship(c)
    c3,s3 = LF_other_relationship(c)
    sc = np.array([s1,s2,s3])
    c = [c1,c2,c3]
    sharp_param = 1.5
    
    prob_sc = np.exp(sc * sharp_param - np.max(sc))
    prob_sc = prob_sc / np.sum(prob_sc)
    #print 'BW:',s1,s2,s3,prob_sc
    if s1==s2 or s3==s1:
        return (0,0)
    return c[np.argmax(prob_sc)],1
    
LFs = [LF_distant_supervision, LF_distant_supervision_last_names,LF_same_last_name,
       LF_and_married, LF_Three_Lists_Between_Words,LF_Three_Lists_Left_Window, LF_no_spouse_in_sentence
      ]

In [ ]:

import numpy as np
import math

def PHI(K,LAMDAi,SCOREi):
    return [K*l*s for (l,s) in zip(LAMDAi,SCOREi)]

def softmax(THETA,LAMDAi,SCOREi):
    x = []
    for k in [1,-1]:
        product = np.dot(PHI(k,LAMDAi,SCOREi),THETA)
        x.append(product)
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def function_conf(THETA,LAMDA,P_cap,Confidence):
    s = 0.0
    i = 0
    for LAMDAi in LAMDA:
        s = s + Confidence[i]*np.dot(np.log(softmax(THETA,LAMDAi)),P_cap[i])
        i = i+1
    return -s

def function(THETA,LAMDA,SCORE,P_cap):
    s = 0.0
    i = 0
    for i in range(len(LAMDA)):
        s = s + np.dot(np.log(softmax(THETA,LAMDA[i],SCORE[i])),P_cap[i])
        i = i+1
    return -s

def P_K_Given_LAMDAi_THETA(K,THETA,LAMDAi,SCOREi):
    x = softmax(THETA,LAMDAi,SCOREi)
    if(K==1):
        return x[0]
    else:
        return x[1]
      

np.random.seed(78)
THETA = np.random.rand(len(LFs),1)

def PHIj(j,K,LAMDAi,SCOREi):
    return LAMDAi[j]*K*SCOREi[j]

def RIGHT(j,LAMDAi,SCOREi,THETA):
    phi = []
    for k in [1,-1]:
        phi.append(PHIj(j,k,LAMDAi,SCOREi))
    x = softmax(THETA,LAMDAi,SCOREi)
    return np.dot(phi,x)
    

def function_conf_der(THETA,LAMDA,P_cap,Confidence):
    der = []
    for j in range(len(THETA)):
        i = 0
        s = 0.0
        for LAMDAi in LAMDA:
            p = 0
            for K in [1,-1]:
                s = s + Confidence[i]*(PHIj(j,K,LAMDAi)-RIGHT(j,LAMDAi,THETA))*P_cap[i][p]
                p = p+1
            i = i+1
        der.append(-s)
    return np.array(der)

def function_der(THETA,LAMDA,SCORE,P_cap):
    der = []
    for j in range(len(THETA)):
        i = 0
        s = 0.0
        for index in range(len(LAMDA)):
            p = 0
            for K in [1,-1]:
                s = s + (PHIj(j,K,LAMDA[index],SCORE[index])-RIGHT(j,LAMDA[index],SCORE[index],THETA))*P_cap[i][p]
                p = p+1
            i = i+1
        der.append(-s)
    return np.array(der)


import numpy as np


def get_LAMDA(cands):
    LAMDA = []
    SCORE = []
    for ci in cands:
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        LAMDA.append(L)
        SCORE.append(S) 
    return LAMDA,SCORE

def get_Confidence(LAMDA):
    confidence = []
    for L in LAMDA:
        Total_L = float(len(L))
        No_zeros = L.count(0)
        No_Non_Zeros = Total_L - No_zeros
        confidence.append(No_Non_Zeros/Total_L)
    return confidence    
    
def get_Initial_P_cap(LAMDA):
    P_cap = []
    for L in LAMDA:
        P_ik = []
        denominator=float(L.count(1)+L.count(-1))
        if(denominator==0):
            denominator=1
        P_ik.append(L.count(1)/denominator)
        P_ik.append(L.count(-1)/denominator)
        P_cap.append(P_ik)
    return P_cap
    #print(np.array(LAMDA))
    #print(np.array(P_cap))append(L)
    #LAMDA=np.array(LAMDA).astype(int)
    #P_cap=np.array(P_cap)
    #print(np.array(LAMDA).shape)
    #print(np.array(P_cap).shape)
    #print(L)
    #print(ci.chemical.get_span(),ci.disease.get_span(),"No.Os",L.count(0),"No.1s",L.count(1),"No.-1s",L.count(-1))
    #print(ci.chemical.get_span(),ci.disease.get_span(),"P(0):",L.count(0)/len(L)," P(1)",L.count(1)/len(L),"P(-1)",L.count(-1)/len(L))

        
def get_P_cap(LAMDA,SCORE,THETA):
    P_cap = []
    for i in range(len(LAMDA)):
        P_capi = softmax(THETA,LAMDA[i],SCORE[i])
        P_cap.append(P_capi)
    return P_cap


def score(predicted_labels,gold_labels):
    tp =0.0
    tn =0.0
    fp =0.0
    fn =0.0
    for i in range(len(gold_labels)):
        if(predicted_labels[i]==gold_labels[i]):
            if(predicted_labels[i]==1):
                tp=tp+1
            else:
                tn=tn+1
        else:
            if(predicted_labels[i]==1):
                fp=fp+1
            else:
                fn=fn+1
    print("tp",tp,"tn",tn,"fp",fp,"fn",fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = (2*precision*recall)/(precision+recall)
    print("precision:",precision)
    print("recall:",recall)
    print("F1 score:",f1score)
                
           
    
from scipy.optimize import minimize
import cPickle as pickle

def get_marginals(P_cap):
    marginals = []
    for P_capi in P_cap:
        marginals.append(P_capi[0])
    return marginals

def predict_labels(marginals):
    predicted_labels=[]
    for i in marginals:
        if(i<0.5):
            predicted_labels.append(-1)
        else:
            predicted_labels.append(1)
    return predicted_labels

def print_details(label,THETA,LAMDA,SCORE):
    print(label)
    P_cap = get_P_cap(LAMDA,SCORE,THETA)
    marginals=get_marginals(P_cap)
    plt.hist(marginals, bins=20)
    plt.show()
    plt.bar(range(0,2796),marginals)
    plt.show()
    predicted_labels=predict_labels(marginals)
    print(len(marginals),len(predicted_labels),len(gold_labels_dev))
    #score(predicted_labels,gold_labels_dev)
    print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary')) 
    
    
    
def train(No_Iter,Use_Confidence=True,theta_file_name="THETA"):
    global THETA
    global dev_LAMDA,dev_SCORE
    LAMDA,SCORE = get_LAMDA(train_cands)
    P_cap = get_Initial_P_cap(LAMDA)
    Confidence = get_Confidence(LAMDA)
    for iteration in range(No_Iter):
        if(Use_Confidence==True):
            res = minimize(function_conf,THETA,args=(LAMDA,P_cap,Confidence), method='BFGS',jac=function_conf_der,options={'disp': True, 'maxiter':20}) #nelder-mead
        else:
            res = minimize(function,THETA,args=(LAMDA,SCORE,P_cap), method='BFGS',jac=function_der,options={'disp': True, 'maxiter':20}) #nelder-mead            
        THETA = res.x # new THETA
        print(THETA)
        P_cap = get_P_cap(LAMDA,SCORE,THETA) #new p_cap 
        print_details("train iteration: "+str(iteration),THETA,dev_LAMDA,dev_SCORE)
        #score(predicted_labels,gold_labels)
    NP_P_cap = np.array(P_cap)
    np.savetxt('Train_P_cap.txt', NP_P_cap, fmt='%f')
    pickle.dump(NP_P_cap,open("Train_P_cap.p","wb"))
    NP_THETA = np.array(THETA)
    np.savetxt(theta_file_name+'.txt', NP_THETA, fmt='%f') 
    pickle.dump( NP_THETA, open( theta_file_name+'.p', "wb" )) # save the file as "outfile_name.npy" 

        
def test(THETA):
    global dev_LAMDA,dev_SCORE
    P_cap = get_P_cap(dev_LAMDA,dev_SCORE,THETA)
    print_details("test:",THETA,dev_LAMDA,dev_SCORE)
    NP_P_cap = np.array(P_cap)
    np.savetxt('Dev_P_cap.txt', NP_P_cap, fmt='%f')
    pickle.dump(NP_P_cap,open("Dev_P_cap.p","wb"))
                    
def load_marginals(s):
    marginals = []
    if(s=="train"):
        train_P_cap = np.load("Train_P_cap.npy")
        marginals = train_P_cap[:,0]
    return marginals



In [19]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for ci in cands:
        L_S_ci=[]
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
    return L_S

def get_L_S(cands):  # sign gives label abs value gives score
    
    L_S = []
    for ci in cands:
        l_s=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            s= (s+1)/2  #to scale scores in [0,1] 
            l_s.append(l*s)
        L_S.append(l_s)
    return L_S

def get_Initial_P_cap_L_S(L_S):
    P_cap = []
    for L,S in L_S:
        P_ik = []
        denominator=float(L.count(1)+L.count(-1))
        if(denominator==0):
            denominator=1
        P_ik.append(L.count(1)/denominator)
        P_ik.append(L.count(-1)/denominator)
        P_cap.append(P_ik)
    return P_cap



In [20]:
import matplotlib.pyplot as plt
   
    
dev_L_S = get_L_S_Tensor(dev_cands)
train_L_S = get_L_S_Tensor(train_cands)


# train_P_cap= get_Initial_P_cap_L_S(train_L_S) 

# dev_P_cap = get_Initial_P_cap_L_S(dev_L_S)


import pickle as pkl

pkl.dump(dev_L_S,open("dev_L_S.p","wb"))
pkl.dump(train_L_S,open("train_L_S.p","wb"))


# pkl.dump(train_P_cap,open("train_P_cap.p","wb"))
# pkl.dump(dev_P_cap,open("dev_P_cap.p","wb"))


In [ ]:
#prepare batch data
# train_L_S_batch,dev_L_S_batch = get_L_S_batch()
# train_P_cap_batch,dev_P_cap_batch = get_P_cap_batch()

In [21]:
from sklearn.metrics import precision_recall_fscore_support

import _pickle as pkl


#pkl.dump(dev_L_S,open("dev_L_S.p","wb"))
#pkl.dump(train_L_S,open("train_L_S.p","wb"))
#pkl.dump(test_L_S,open("test_L_S.p","wb"))

#pkl.dump(train_P_cap,open("train_P_cap.p","wb"))
#pkl.dump(dev_P_cap,open("dev_P_cap.p","wb"))
#pkl.dump(test_P_cap,open("test_P_cap.p","wb"))

dev_L_S = pkl.load( open( "dev_L_S.p", "rb" ) )
train_L_S = pkl.load( open( "train_L_S.p", "rb" ) )
# test_L_S = pkl.load( open( "test_L_S.p", "rb" ) )

# train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
# dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )
# test_P_cap = pkl.load( open( "test_P_cap.p", "rb" ) )

def get_L_S_batch():
    dev_L_batch = []
    dev_S_batch = []
    dev_L_S_batch = []
    train_L_batch = []
    train_S_batch = []
    train_L_S_batch = []
    for l,s in train_L_S:
        train_L_batch.append(l)
        train_S_batch.append(s)
    train_L_S_batch = [train_L_batch, train_S_batch]
    for l,s in dev_L_S:
        dev_L_batch.append(l)
        dev_S_batch.append(s)
    dev_L_S_batch = [dev_L_batch, dev_S_batch]
    return train_L_S_batch,dev_L_S_batch


def get_P_cap_batch():
    kp1_train= []
    kn1_train = []
    kp1_dev= []
    kn1_dev = []
    for pci in train_P_cap:
        kp1_train.append(pci[0])
        kn1_train.append(pci[1])
    for pci in dev_P_cap:
        kp1_dev.append(pci[0])
        kn1_dev.append(pci[1])
    return [kp1_train,kn1_train],[kp1_dev,kn1_dev]
        
def get_mini_batches(X,P_cap,bsize): #X : (train/dev/)_L_S_batch
    for i in range(0, len(X[0]) - bsize + 1, bsize):
        indices = slice(i, i + bsize)
        #print(indices)
        yield [X[0][indices],X[1][indices]],P_cap[indices]

# train_L_S_batch,dev_L_S_batch = get_L_S_batch()

#for x in get_mini_batches(train_L_S_batch,200):
#    print(len(x),len(x[0]),len(x[0][0]))
    


In [ ]:
#stochastic + cross entropy logits func + remove min(theta,0) in loss
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

result_dir = "./"
config = projector.ProjectorConfig()
tf.logging.set_verbosity(tf.logging.INFO)
summary_writer = tf.summary.FileWriter(result_dir)
projector.visualize_embeddings(summary_writer, config)

tf.reset_default_graph()

train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )


dim = 2 #(labels,scores)

_x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
_p_cap = tf.placeholder(tf.float64,shape=(2))

alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                        dtype=tf.float64)

thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                        dtype=tf.float64)

#print([n.name for n in tf.get_default_graph().as_graph_def().node])

#for k = 1

k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

k_n1 = tf.negative(k_p1)


l,s = tf.unstack(_x)

prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


mul_L_S = tf.multiply(l,prelu_out_s)

phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))

phi_out = tf.stack([phi_p1,phi_n1])

# loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
#         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
#         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
#         - tf.minimum( tf.reduce_min(thetas),0)

loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=phi_out,labels=_p_cap))) + \
        tf.reduce_sum(tf.multiply(alphas,alphas))\
        + tf.reduce_sum(tf.multiply(thetas,thetas)) 
        #- tf.minimum( tf.reduce_min(thetas),0)
         

predict = tf.argmax(tf.nn.softmax(phi_out))

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

new_p_cap = tf.nn.softmax(phi_out)

check_op = tf.add_check_numerics_ops()

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
'''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
        _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        print(_los)
        print(_l)
        print(_s)
        print(_a)
        print(_os)        
        print(_t)
        print()'''
    
for i in range(10):
    c = 0
    te_prev=1
    total_te = 0
    for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
        
        a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        total_te+=te_curr
        #print(a)
        #print(t)
        #print
        New_P_cap = []
        newPcap = sess.run(new_p_cap,feed_dict={_x:train_L_S[c+1],_p_cap:train_P_cap[c+1]})
        train_P_cap[c+1] = newPcap
#         for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
#             newPcap = sess.run(new_p_cap,feed_dict={_x:L_S_i,_p_cap:P_cap_i})
#             New_P_cap.append(newPcap)
#         train_P_cap = New_P_cap

        
        if(abs(te_curr-te_prev)<1e-300):
            predicted_labels = []
            for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                predicted_labels.append(p)
            print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
            break
        
#         if(c%20==0):
#             predicted_labels = []
#             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
#                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
#                 predicted_labels.append(p)
#             print(predicted_labels.count(0),predicted_labels.count(1))
#             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
#         c+=1
        te_prev = te_curr
    
    pl = []
    for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
        te_curr,p = sess.run([loss,predict],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        pl.append(p)
    predicted_labels = [-1 if x==0 else x for x in pl]
    print(total_te)
    print(predicted_labels.count(1),predicted_labels.count(-1))
    print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
      
    


In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score

def class_wise(y_test,y_score):
    print(average_precision_score(y_test, y_score, average=None))
    
def draw_curve(y_test,y_score):
    average_precision = average_precision_score(y_test, y_score)

    precision, recall, _ = precision_recall_curve(y_test, y_score)

    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class Precision-Recall curve: AUC={0:0.2f}'.format(
              average_precision))

In [ ]:
#stochastic + cross entropy logits func + remove min(theta,0) in loss + precision recall curve
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

result_dir = "./"
config = projector.ProjectorConfig()
tf.logging.set_verbosity(tf.logging.INFO)
summary_writer = tf.summary.FileWriter(result_dir)
projector.visualize_embeddings(summary_writer, config)

tf.reset_default_graph()

train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )


dim = 2 #(labels,scores)

_x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
_p_cap = tf.placeholder(tf.float64,shape=(2))

alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                        dtype=tf.float64)

thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                        dtype=tf.float64)

#print([n.name for n in tf.get_default_graph().as_graph_def().node])

#for k = 1

k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

k_n1 = tf.negative(k_p1)


l,s = tf.unstack(_x)

prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


mul_L_S = tf.multiply(l,prelu_out_s)

phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))

phi_out = tf.stack([phi_p1,phi_n1])

# loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
#         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
#         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
#         - tf.minimum( tf.reduce_min(thetas),0)

loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=phi_out,labels=_p_cap))) + \
        tf.reduce_sum(tf.multiply(alphas,alphas))\
        + tf.reduce_sum(tf.multiply(thetas,thetas)) 
        #- tf.minimum( tf.reduce_min(thetas),0)
         

predict = tf.argmax(tf.nn.softmax(phi_out))

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

new_p_cap = tf.nn.softmax(phi_out)

check_op = tf.add_check_numerics_ops()

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
'''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
        _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        print(_los)
        print(_l)
        print(_s)
        print(_a)
        print(_os)        
        print(_t)
        print()'''
    
for i in range(10):
    c = 0
    te_prev=1
    total_te = 0
    for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
        
        a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        total_te+=te_curr
        #print(a)
        #print(t)
        #print
        New_P_cap = []
        newPcap = sess.run(new_p_cap,feed_dict={_x:train_L_S[c+1],_p_cap:train_P_cap[c+1]})
        train_P_cap[c+1] = newPcap
#         for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
#             newPcap = sess.run(new_p_cap,feed_dict={_x:L_S_i,_p_cap:P_cap_i})
#             New_P_cap.append(newPcap)
#         train_P_cap = New_P_cap

        
        if(abs(te_curr-te_prev)<1e-300):
            break
            predicted_labels = []
            for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                predicted_labels.append(p)
            print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
            
        
#         if(c%20==0):
#             predicted_labels = []
#             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
#                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
#                 predicted_labels.append(p)
#             print(predicted_labels.count(0),predicted_labels.count(1))
#             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
#         c+=1
        te_prev = te_curr
    
    pl = []
    probs = []
    for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
        te_curr,p,prob = sess.run([loss,predict,new_p_cap],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        pl.append(p)
        probs.append(prob)
    probs = np.array(probs)
    predicted_labels = [-1 if x==0 else x for x in pl]
    print(i,total_te)
    print(predicted_labels.count(1),predicted_labels.count(-1))
    draw_curve(np.array(gold_labels_dev),probs[:,1])
    print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
      
    


In [ ]:
#stochastic + cross entropy logits func + weighted logits+remove min(theta,0) in loss 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN(weight):
    print()
    print("weight: ",weight)
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)
    projector.visualize_embeddings(summary_writer, config)

    tf.reset_default_graph()

    train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
    dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )

    
    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
    _p_cap = tf.placeholder(tf.float64,shape=(2))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                            dtype=tf.float64)

    #print([n.name for n in tf.get_default_graph().as_graph_def().node])

    #for k = 1

    k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

    k_n1 = tf.negative(k_p1)


    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))

    phi_out = tf.stack([phi_p1,phi_n1])
    
    class_weights = tf.constant([weight, 1.0 - weight],dtype=tf.float64)
    
    weighted_logits = tf.multiply(phi_out, class_weights) 

    # loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
    #         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
    #         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
    #         - tf.minimum( tf.reduce_min(thetas),0)
    
    loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=_p_cap,logits=weighted_logits))) + \
            tf.reduce_sum(tf.multiply(alphas,alphas))\
            + tf.reduce_sum(tf.multiply(thetas,thetas)) 
            #- tf.minimum( tf.reduce_min(thetas),0)


    predict = tf.argmax(tf.nn.softmax(weighted_logits))

    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

    new_p_cap = tf.nn.softmax(phi_out)

    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    '''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
            _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            print(_los)
            print(_l)
            print(_s)
            print(_a)
            print(_os)        
            print(_t)
            print()'''

    for i in range(1):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            total_te+=te_curr
            #print(a)
            #print(t)
            #print
            New_P_cap = []
            newPcap = sess.run(new_p_cap,feed_dict={_x:train_L_S[c+1],_p_cap:train_P_cap[c+1]})
            train_P_cap[c+1] = newPcap
    #         for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
    #             newPcap = sess.run(new_p_cap,feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #             New_P_cap.append(newPcap)
    #         train_P_cap = New_P_cap


            if(abs(te_curr-te_prev)<1e-300):
                break
                predicted_labels = []
                for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                    de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                    predicted_labels.append(p)
                print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))


    #         if(c%20==0):
    #             predicted_labels = []
    #             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
    #                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #                 predicted_labels.append(p)
    #             print(predicted_labels.count(0),predicted_labels.count(1))
    #             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
    #         c+=1
            te_prev = te_curr

        pl = []
        probs = []
        for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
            te_curr,p,prob = sess.run([loss,predict,new_p_cap],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            pl.append(p)
            probs.append(prob)
        probs = np.array(probs)
        predicted_labels = [-1 if x==0 else x for x in pl]
        print(i,total_te)
        print(predicted_labels.count(1),predicted_labels.count(-1))
        print("class wise:")
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average=None))
        #draw_curve(np.array(gold_labels_dev),probs[:,1])
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
        print('macro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
        print('micro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='micro'))

    

for i in np.linspace(0,1,5):
    train_NN(i)

In [ ]:
# 2 stochastic + cross entropy logits func + weighted logits+remove min(theta,0) in loss 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN(weight):
    print()
    print("weight: ",weight)
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)
    projector.visualize_embeddings(summary_writer, config)

    tf.reset_default_graph()

    train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
    dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )

    
    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
    _p_cap = tf.placeholder(tf.float64,shape=(2))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                            dtype=tf.float64)

    #print([n.name for n in tf.get_default_graph().as_graph_def().node])

    #for k = 1

    k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

    k_n1 = tf.negative(k_p1)


    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))

    phi_out = tf.stack([phi_p1,phi_n1])
    
    class_weights = tf.constant([weight, 1.0 - weight],dtype=tf.float64)
    
    weighted_logits = tf.multiply(phi_out, class_weights) 

    # loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
    #         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
    #         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
    #         - tf.minimum( tf.reduce_min(thetas),0)
    
    loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=_p_cap,logits=weighted_logits))) + \
            tf.reduce_sum(tf.multiply(alphas,alphas))\
            + tf.reduce_sum(tf.multiply(thetas,thetas)) 
            #- tf.minimum( tf.reduce_min(thetas),0)


    predict = tf.argmax(tf.nn.softmax(weighted_logits))

    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

    new_p_cap = tf.nn.softmax(phi_out)

    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    '''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
            _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            print(_los)
            print(_l)
            print(_s)
            print(_a)
            print(_os)        
            print(_t)
            print()'''

    for i in range(1):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            total_te+=te_curr
            #print(a)
            #print(t)
            #print
            New_P_cap = []
            newPcap = sess.run(new_p_cap,feed_dict={_x:train_L_S[c+1],_p_cap:train_P_cap[c+1]})
            train_P_cap[c+1] = newPcap
    #         for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
    #             newPcap = sess.run(new_p_cap,feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #             New_P_cap.append(newPcap)
    #         train_P_cap = New_P_cap


            if(abs(te_curr-te_prev)<1e-300):
                break
                predicted_labels = []
                for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                    de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                    predicted_labels.append(p)
                print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))


    #         if(c%20==0):
    #             predicted_labels = []
    #             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
    #                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #                 predicted_labels.append(p)
    #             print(predicted_labels.count(0),predicted_labels.count(1))
    #             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
    #         c+=1
            te_prev = te_curr

        pl = []
        probs = []
        for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
            te_curr,p,prob = sess.run([loss,predict,new_p_cap],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            pl.append(p)
            probs.append(prob)
        probs = np.array(probs)
        predicted_labels = [-1 if x==0 else x for x in pl]
        print(i,total_te)
        print(predicted_labels.count(1),predicted_labels.count(-1))
        print("class wise:")
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average=None))
        #draw_curve(np.array(gold_labels_dev),probs[:,1])
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
        print('macro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
        print('micro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='micro'))

    

for i in np.linspace(0,1,10):
    train_NN(i)

In [ ]:
# 2 find thresh + stochastic + cross entropy logits func + weighted logits+remove min(theta,0) in loss 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN(weight):
    print()
    print("weight: ",weight)
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)
    projector.visualize_embeddings(summary_writer, config)

    tf.reset_default_graph()

    train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
    dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )

    
    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
    _p_cap = tf.placeholder(tf.float64,shape=(2))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                            dtype=tf.float64)

    #print([n.name for n in tf.get_default_graph().as_graph_def().node])

    #for k = 1

    k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

    k_n1 = tf.negative(k_p1)


    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))

    phi_out = tf.stack([phi_p1,phi_n1])
    
    class_weights = tf.constant([weight, 1.0 - weight],dtype=tf.float64)
    
    weighted_logits = tf.multiply(phi_out, class_weights) 

    # loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
    #         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
    #         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
    #         - tf.minimum( tf.reduce_min(thetas),0)
    
    loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=_p_cap,logits=weighted_logits))) 
            
            #- tf.minimum( tf.reduce_min(thetas),0)


    predict = tf.argmax(tf.nn.softmax(weighted_logits))

    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

    new_p_cap = tf.nn.softmax(phi_out)

    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    '''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
            _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            print(_los)
            print(_l)
            print(_s)
            print(_a)
            print(_os)        
            print(_t)
            print()'''

    for i in range(1):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            total_te+=te_curr
            #print(a)
            #print(t)
            #print
            New_P_cap = []
            newPcap = sess.run(new_p_cap,feed_dict={_x:train_L_S[c+1],_p_cap:train_P_cap[c+1]})
            train_P_cap[c+1] = newPcap
    #         for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
    #             newPcap = sess.run(new_p_cap,feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #             New_P_cap.append(newPcap)
    #         train_P_cap = New_P_cap


            if(abs(te_curr-te_prev)<1e-300):
                break
                predicted_labels = []
                for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                    de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                    predicted_labels.append(p)
                print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))


    #         if(c%20==0):
    #             predicted_labels = []
    #             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
    #                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #                 predicted_labels.append(p)
    #             print(predicted_labels.count(0),predicted_labels.count(1))
    #             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
    #         c+=1
            te_prev = te_curr

        pl = []
        probs = []
        tuned_alphas = []
        tuned_thetas = []
        for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
            tuned_alphas,tuned_thetas,te_curr,p,prob = sess.run([alphas,thetas,loss,predict,new_p_cap],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            pl.append(p)
            probs.append(prob)
        probs = np.array(probs)
        predicted_labels = [-1 if x==0 else x for x in pl]
        print(i,total_te)
        print(tuned_alphas)
        print(tuned_thetas)
        print(predicted_labels.count(1),predicted_labels.count(-1))
        print("class wise:")
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average=None))
        #draw_curve(np.array(gold_labels_dev),probs[:,1])
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
        print('macro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
        print('micro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='micro'))

    

for i in np.linspace(0,1,10):
    train_NN(i)

In [ ]:
# 8/12 find thresh + remove regular on alphas + stochastic + cross entropy logits func + weighted logits+remove min(theta,0) in loss 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN(weight,reg):
    print()
    print("weight: ",weight)
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)
    projector.visualize_embeddings(summary_writer, config)

    tf.reset_default_graph()

    REG_CONST = tf.constant(reg,dtype=tf.float64)
    train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
    dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )

    
    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
    _p_cap = tf.placeholder(tf.float64,shape=(2))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                            dtype=tf.float64)

    #print([n.name for n in tf.get_default_graph().as_graph_def().node])

    #for k = 1

    k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

    k_n1 = tf.negative(k_p1)


    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))

    phi_out = tf.stack([phi_p1,phi_n1])
    
    class_weights = tf.constant([weight, 1.0 - weight],dtype=tf.float64)
    
    weighted_logits = tf.multiply(phi_out, class_weights) 

    # loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
    #         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
    #         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
    #         - tf.minimum( tf.reduce_min(thetas),0)
    
    loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=_p_cap,logits=weighted_logits))) \
            + (REG_CONST * tf.reduce_sum(tf.multiply(thetas,thetas))) \
            + (REG_CONST * tf.reduce_sum(tf.multiply(alphas,alphas))) \
            #- tf.minimum( tf.reduce_min(thetas),0)


    predict = tf.argmax(tf.nn.softmax(weighted_logits))

    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

    new_p_cap = tf.nn.softmax(phi_out)

    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    '''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
            _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            print(_los)
            print(_l)
            print(_s)
            print(_a)
            print(_os)        
            print(_t)
            print()'''

    for i in range(1):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            total_te+=te_curr
            #print(a)
            #print(t)
            #print
            New_P_cap = []
            newPcap = sess.run(new_p_cap,feed_dict={_x:train_L_S[c+1],_p_cap:train_P_cap[c+1]})
            train_P_cap[c+1] = newPcap
    #         for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
    #             newPcap = sess.run(new_p_cap,feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #             New_P_cap.append(newPcap)
    #         train_P_cap = New_P_cap


            if(abs(te_curr-te_prev)<1e-300):
                break
                predicted_labels = []
                for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                    de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                    predicted_labels.append(p)
                print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels)))


    #         if(c%20==0):
    #             predicted_labels = []
    #             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
    #                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #                 predicted_labels.append(p)
    #             print(predicted_labels.count(0),predicted_labels.count(1))
    #             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
    #         c+=1
            te_prev = te_curr

        pl = []
        probs = []
        tuned_alphas = []
        for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
            tuned_alphas,te_curr,p,prob = sess.run([alphas,loss,predict,new_p_cap],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            pl.append(p)
            probs.append(prob)
        probs = np.array(probs)
        predicted_labels = [-1 if x==0 else x for x in pl]
        print(i,total_te)
        print(tuned_alphas)
        print(predicted_labels.count(1),predicted_labels.count(-1))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
        p,r,f,_ = precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary')
        print("ret",p,r,f)
        return (p,r)
        #draw_pr_curve(precisionArr,recallArr,reg)
        #draw_curve(np.array(gold_labels_dev),probs[:,1])
        


In [ ]:

precisionArr = []
recallArr = []
def draw_pr_curve(reg):
    print("lambda",reg)
    print("precision",precisionArr)
    print("recall",recallArr)
    plt.step(recallArr, precisionArr, color='b', alpha=0.2,where='post')
    plt.fill_between(recallArr, precisionArr, step='post', alpha=0.2,color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0, 1])
    plt.xlim([0, 1])
    plt.title('Precision-Recall curve')
    
for reg in [1.2,1,4]:
    for i in np.linspace(0,1,10):
        p,r = train_NN(i,reg)
        precisionArr.append(p)
        recallArr.append(r)
    draw_pr_curve(reg)

In [ ]:
for reg in [0.6,0.8]:
    for i in np.linspace(0,1,10):
        p,r = train_NN(i,reg)
        precisionArr.append(p)
        recallArr.append(r)
    draw_pr_curve(reg)

In [ ]:
precision = [0.164,0.164,0.224,0.234,0.234,0.233,0.233,0.233,0.233,0.233]
recall = [0.607,0.607,0.561,0.551,0.551,0.551,0.551,0.551,0.551,0.551]
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 0.25])
plt.xlim([0.55, 0.61])
plt.title('Precision-Recall curve')

In [ ]:
# 8/12 find thresh + remove regular on alphas + stochastic + cross entropy logits func + weighted logits+remove min(theta,0) in loss 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN(weight,reg):
    print()
    print("weight: ",weight)
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)
    projector.visualize_embeddings(summary_writer, config)

    tf.reset_default_graph()

    REG_CONST = tf.constant(reg,dtype=tf.float64)
    train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
    dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )

    
    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
    _p_cap = tf.placeholder(tf.float64,shape=(2))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                            dtype=tf.float64)

    #print([n.name for n in tf.get_default_graph().as_graph_def().node])

    #for k = 1

    k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

    k_n1 = tf.negative(k_p1)


    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))

    phi_out = tf.stack([phi_p1,phi_n1])
    
    class_weights = tf.constant([weight, 1.0 - weight],dtype=tf.float64)
    
    weighted_logits = tf.multiply(phi_out, class_weights) 

    # loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
    #         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
    #         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
    #         - tf.minimum( tf.reduce_min(thetas),0)
    
    loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=_p_cap,logits=weighted_logits))) \
            + (REG_CONST * tf.reduce_sum(tf.multiply(thetas,thetas))) \
            #+tf.reduce_sum(tf.multiply(alphas,alphas)) \
            #- tf.minimum( tf.reduce_min(thetas),0)


    predict = tf.argmax(tf.nn.softmax(weighted_logits))

    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

    new_p_cap = tf.nn.softmax(phi_out)

    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    '''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
            _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            print(_los)
            print(_l)
            print(_s)
            print(_a)
            print(_os)        
            print(_t)
            print()'''

    for i in range(1):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            total_te+=te_curr
            #print(a)
            #print(t)
            #print
            New_P_cap = []
            newPcap = sess.run(new_p_cap,feed_dict={_x:train_L_S[c+1],_p_cap:train_P_cap[c+1]})
            train_P_cap[c+1] = newPcap
    #         for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
    #             newPcap = sess.run(new_p_cap,feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #             New_P_cap.append(newPcap)
    #         train_P_cap = New_P_cap


            if(abs(te_curr-te_prev)<1e-300):
                break
                predicted_labels = []
                for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                    de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                    predicted_labels.append(p)
                print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels)))


    #         if(c%20==0):
    #             predicted_labels = []
    #             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
    #                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
    #                 predicted_labels.append(p)
    #             print(predicted_labels.count(0),predicted_labels.count(1))
    #             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
    #         c+=1
            te_prev = te_curr

        pl = []
        probs = []
        tuned_alphas = []
        for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
            tuned_alphas,te_curr,p,prob = sess.run([alphas,loss,predict,new_p_cap],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
            pl.append(p)
            probs.append(prob)
        probs = np.array(probs)
        predicted_labels = [-1 if x==0 else x for x in pl]
        print(i,total_te)
        print(tuned_alphas)
        print(predicted_labels.count(1),predicted_labels.count(-1))
        #print("class wise:")
        #print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average=None))
        p,r,f,_ = precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average=None)
        precisionArr.append(p)
        recallArr.append(r)
        #draw_curve(np.array(gold_labels_dev),probs[:,1])
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
        #print('macro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
        #print('micro',precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='micro'))

    
for reg in [0.5,1,1.5]:
    for i in np.linspace(0,1,10):
        train_NN(i,reg)
        draw_pr_curve(precisionArr,recallArr,reg)

In [ ]:
#stochastic + cross entropy logits func + yi fixed to output of model on discrete lfs
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

result_dir = "./"
config = projector.ProjectorConfig()
tf.logging.set_verbosity(tf.logging.INFO)
summary_writer = tf.summary.FileWriter(result_dir)
projector.visualize_embeddings(summary_writer, config)

tf.reset_default_graph()

train_P_cap = np.load("Train_P_cap.npy")


#print(train_P_cap)

# discrete_labels = predict_labels(train_P_cap[:1])

# for i in range of discrete_labels:
#     print(train_P_cap[i],discrete_labels[i])

#train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )


dim = 2 #(labels,scores)

_x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))
_p_cap = tf.placeholder(tf.float64,shape=(2))

#W =tf.Variable(tf.truncated_normal([len(LFs),len(LFs)], stddev=0.8,dtype = tf.float64))

#b =tf.Variable(tf.truncated_normal([len(LFs)], stddev=0.01,dtype = tf.float64))


alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                        dtype=tf.float64)

thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.0),
                        dtype=tf.float64)

#print([n.name for n in tf.get_default_graph().as_graph_def().node])

#for k = 1

k_p1 = tf.ones(shape=(dim,len(LFs)),dtype=tf.float64)

k_n1 = tf.negative(k_p1)


l,s = tf.unstack(_x)

prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


mul_L_S = tf.multiply(l,prelu_out_s)

# phi_out = tf.multiply(l,prelu_out_s)

phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(mul_L_S,k_n1),thetas))


# additional_layer_out = tf.matmul(tf.expand_dims(mul_L_S,0),W) + b

# phi_p1 = tf.reduce_sum(tf.multiply(tf.squeeze(additional_layer_out),thetas))

# phi_n1 = tf.reduce_sum(tf.multiply(tf.multiply(tf.squeeze(additional_layer_out),k_n1),thetas))

phi_out = tf.stack([phi_p1,phi_n1])


# loss = tf.negative(tf.reduce_sum(tf.multiply(tf.log(tf.nn.softmax(phi_out)),_p_cap))) + \
#         tf.reduce_sum(tf.multiply(alphas,alphas)) + \
#         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
#         - tf.minimum( tf.reduce_min(thetas),0)

loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=phi_out,labels=_p_cap))) + \
        tf.reduce_sum(tf.multiply(alphas,alphas))\
        + tf.reduce_sum(tf.multiply(thetas,thetas)) 
        #- tf.minimum( tf.reduce_min(thetas),0)\
         

predict = tf.argmax(tf.nn.softmax(phi_out))

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

new_p_cap = tf.nn.softmax(phi_out)

check_op = tf.add_check_numerics_ops()

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
'''for L_S_i,P_cap_i in zip(train_L_S,dev_P_cap):
        _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        print(_los)
        print(_l)
        print(_s)
        print(_a)
        print(_os)        
        print(_t)
        print()'''
    
for i in range(10):
    c = 0
    te_prev=1
    total_te = 0
    for L_S_i,P_cap_i in zip(train_L_S,train_P_cap):
        
        a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        total_te+=te_curr
        #print(a)
        #print(t)
        print
        if(abs(te_curr-te_prev)<1e-300):
            predicted_labels = []
            for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
                de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
                predicted_labels.append(p)
            print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
            break
        
#         if(c%20==0):
#             predicted_labels = []
#             for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
#                 de_curr,p,_ = sess.run([loss,predict,train_step],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
#                 predicted_labels.append(p)
#             print(predicted_labels.count(0),predicted_labels.count(1))
#             print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
#         c+=1
        te_prev = te_curr
    pl = []
    print(total_te)
    for L_S_i,P_cap_i in zip(dev_L_S,dev_P_cap):
        te_curr,p = sess.run([loss,predict],feed_dict={_x:L_S_i,_p_cap:P_cap_i})
        pl.append(p)
    predicted_labels = [-1 if x==0 else x for x in pl]
    print(predicted_labels.count(1),predicted_labels.count(-1))
    print(c," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
      
    


In [ ]:
# Batch with cross entropy logits function  + additional layer
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

result_dir = "./"
config = projector.ProjectorConfig()
tf.logging.set_verbosity(tf.logging.INFO)
summary_writer = tf.summary.FileWriter(result_dir)
projector.visualize_embeddings(summary_writer, config)

tf.reset_default_graph()


dim = 2 #(labels,scores)

data_size = len(train_L_S_batch[0])

dev_data_size = len(dev_L_S_batch[0])

train_P_cap = pkl.load( open( "train_P_cap.p", "rb" ) )
dev_P_cap = pkl.load( open( "dev_P_cap.p", "rb" ) )


#train_P_cap = np.full([data_size,2],0.5)

#print(train_P_cap)
#print(train_P_cap.shape)
#dev_P_cap = np.full([dev_data_size,2],0.5)



print(data_size)

_x = tf.placeholder(tf.float64,shape=(dim,None,len(LFs)))
_p_cap = tf.placeholder(tf.float64,shape=(None,2))

W =tf.Variable(tf.truncated_normal([len(LFs),len(LFs)], stddev=0.8,dtype = tf.float64))

b =tf.Variable(tf.truncated_normal([len(LFs)], stddev=0.01,dtype = tf.float64))

alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                        dtype=tf.float64)

thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(0.01),
                        dtype=tf.float64)

#print([n.name for n in tf.get_default_graph().as_graph_def().node])



l,s = tf.unstack(_x)

prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        


mul_L_S = tf.multiply(l,prelu_out_s)

additional_layer_out = tf.matmul(mul_L_S,W) + b

phi_p1 = tf.matmul(additional_layer_out,tf.expand_dims(thetas,-1))

phi_n1 = tf.matmul(tf.negative(additional_layer_out),tf.expand_dims(thetas,-1))



# phi_p1 = tf.matmul(mul_L_S,tf.expand_dims(thetas,-1))

# phi_n1 = tf.matmul(tf.negative(mul_L_S),tf.expand_dims(thetas,-1))


phi_out = tf.concat([phi_p1,phi_n1],1)

# pio = tf.Print(phi_out,[phi_out])

# loss = tf.negative(tf.reduce_sum(tf.matmul(tf.transpose(tf.log(tf.nn.softmax(phi_out))),_p_cap))) + \
#         tf.reduce_sum(tf.multiply(alphas,alphas)) +\
#         tf.reduce_sum(tf.multiply(thetas,thetas)) +\
#         - tf.minimum( tf.reduce_min(thetas),0)


loss = tf.negative(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=phi_out,labels=_p_cap))) + \
        tf.reduce_sum(tf.multiply(alphas,alphas))\
        - tf.minimum( tf.reduce_min(thetas),0)\
         + tf.reduce_sum(tf.multiply(thetas,thetas)) 
        

predict = tf.argmax(tf.nn.softmax(phi_out),1)

predict_2 = tf.where(tf.greater(tf.slice(tf.nn.softmax(phi_out),[0,1],[dev_data_size,1]),0.5),
                    tf.ones((dev_data_size,1)),tf.negative(tf.ones((dev_data_size,1))))

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 

new_p_cap = tf.nn.softmax(phi_out)

check_op = tf.add_check_numerics_ops()

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

#print(sess.run([phi_out,predict],feed_dict={_x:train_L_S_batch,_p_cap:train_P_cap}))

# for i in range(100):
#     _l,_s,_os,_a,_t,_los,_ = sess.run([l,s,prelu_out_s,alphas,thetas,loss,train_step],feed_dict={_x:train_L_S_batch,_p_cap:train_P_cap})
#     print(_los)
#     print(_l)
#     print(_s)
#     print(_a)
#     print(_os)        
#     print(_t)
#     print()

for i in range(100):
    c = 0
    te_prev=1
    total_te = 0
    a,t,te_curr,_, = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:train_L_S_batch,_p_cap:train_P_cap})
    print(te_curr)
    
    train_P_cap = sess.run(new_p_cap,feed_dict={_x:train_L_S_batch,_p_cap:train_P_cap}) 
    #print(train_P_cap[0:5])
    #print(a)
    #print(t)
    #print()   
    if(i%1 == 0):
        te_curr,pl,pl2,_ = sess.run([loss,predict,predict_2,train_step],feed_dict={_x:dev_L_S_batch,_p_cap:dev_P_cap})
        pl2 = pl2.flatten().tolist()
        pl = pl.flatten().tolist()
        #print(te_curr)
        #predicted_labels = pl2
        predicted_labels = [-1 if x==0 else 1 for x in pl]
        #for l,l2 in zip(predicted_labels,pl2):
        #    print(l,l2)
        
        print(predicted_labels.count(1),predicted_labels.count(-1))
        #print(predicted_labels,gold_labels_dev)
        print(i," ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
 
    if(abs(te_curr-te_prev)<1e-20):
          break
            
  

In [ ]:
# All LF_Threshold =0.3 and softmax_Threshold=0.3 ,to be run
train(2,Use_Confidence=False,theta_file_name="THETA")

test(THETA)

In [ ]:
def print_details(label,THETA,LAMDA,SCORE):
    print(label)
    P_cap = get_P_cap(LAMDA,SCORE,THETA)
    marginals=get_marginals(P_cap)
    plt.hist(marginals, bins=20)
    plt.show()
    #plt.bar(range(0,2796),marginals)
    #plt.show()
    predicted_labels=predict_labels(marginals)
    print(len(marginals),len(predicted_labels),len(gold_labels_dev))
    #score(predicted_labels,gold_labels_dev)
    print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary')) 
    
def predict_labels(marginals):
    predicted_labels=[]
    for i in marginals:
        if(i<0.5):
            predicted_labels.append(-1)
        else:
            predicted_labels.append(1)
    return predicted_labels

#import cPickle as pickle
#THETA = pickle.load( open( "THETA.p", "rb" ) )
#test(THETA)
#LAMDA,SCORE = get_LAMDA(dev_cands)
#Confidence = get_Confidence(LAMDA)

#P_cap = get_P_cap(LAMDA,SCORE,THETA)
#marginals=get_marginals(P_cap)
#plt.hist(marginals, bins=20)
#plt.show()
#plt.bar(range(0,888),train_marginals)
#plt.show()

print_details("dev set",THETA,dev_LAMDA,dev_SCORE)
predicted_labels=predict_labels(marginals)


sorted_predicted_labels=[x for (y,x) in sorted(zip(Confidence,predicted_labels))] #sort Labels as per Confidence
sorted_predicted_labels=list(reversed(sorted_predicted_labels))


for i,j in enumerate(reversed(sorted(zip(Confidence,predicted_labels,gold_labels_dev)))):
    if i>20:
        break
    print i,j
#print(len(marginals),len(predicted_labels),len(gold_labels_dev))
#no_of_labels=186#int(len(predicted_labels)*0.1)  #54 - >0.2  , 108>= 0.15 , 186>= 0.12
#print(len(sorted_predicted_labels[0:no_of_labels]))
no_of_labels=2796
score(predicted_labels[0:no_of_labels],gold_labels_dev[0:no_of_labels])


In [22]:
#stochastic + weighted cross entropy logits func + remove min(theta,0) in loss -- Marked
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN():
    print()
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)

    tf.reset_default_graph()

    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(1),
                            dtype=tf.float64)

    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        

    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.negative(mul_L_S),thetas))

    phi_out = tf.stack([phi_n1,phi_p1])
    
    predict = tf.argmax(tf.nn.softmax(phi_out))

    loss = tf.negative(tf.reduce_logsumexp(phi_out))

    train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(loss) 


    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for i in range(1):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i in train_L_S:

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i})
            total_te+=te_curr

            if(abs(te_curr-te_prev)<1e-200):
                break

            if(c%4000==0):
                pl = []
                for L_S_i in dev_L_S:
                    a,t,de_curr,p = sess.run([alphas,thetas,loss,predict],feed_dict={_x:L_S_i})
                    pl.append(p)
                predicted_labels = [-1 if x==0 else x for x in pl]
                print()
                print(total_te/4000)
                total_te=0
                print(a)
                print(t)
#                 print()
                print(predicted_labels.count(-1),predicted_labels.count(1))
                print(c," d ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
            c+=1
            te_prev = te_curr
        pl = []
        for L_S_i in dev_L_S:
            p = sess.run(predict,feed_dict={_x:L_S_i})
            pl.append(p)
        predicted_labels = [-1 if x==0 else x for x in pl]
        print(i,total_te)
        print(predicted_labels.count(-1),predicted_labels.count(1))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
    
train_NN()



-0.00017328679513998633
2313 501
0  d  (0.3790162864931616, 0.4733986475956355, 0.3991855477883548, None)


/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 -0.6931471805599453
2313 501
(0.3790162864931616, 0.4733986475956355, 0.3991855477883548, None)


In [49]:
#stochastic + weighted cross entropy logits func + remove min(theta,0) in loss 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN():
    print()
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)

    tf.reset_default_graph()

    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer= tf.truncated_normal_initializer(0.2,0.1),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer= tf.truncated_normal_initializer(1,0.1),
                            dtype=tf.float64)

    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        

    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.negative(mul_L_S),thetas))

    phi_out = tf.stack([phi_n1,phi_p1])
    
    predict = tf.argmax(tf.nn.softmax(phi_out))

    loss = tf.negative(tf.reduce_logsumexp(phi_out))

    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 


    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for i in range(5):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i in train_L_S:

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i})
            total_te+=te_curr

            if(abs(te_curr-te_prev)<1e-200):
                break

            if(c%4000==0):
                pl = []
                for L_S_i in dev_L_S:
                    a,t,de_curr,p = sess.run([alphas,thetas,loss,predict],feed_dict={_x:L_S_i})
                    pl.append(p)
                predicted_labels = [-1 if x==0 else x for x in pl]
                print()
                print(total_te/4000)
                total_te=0
                print(a)
                print(t)
#                 print()
                print(predicted_labels.count(-1),predicted_labels.count(1))
                print(c," d ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
            c+=1
            te_prev = te_curr
        pl = []
        for L_S_i in dev_L_S:
            p = sess.run(predict,feed_dict={_x:L_S_i})
            pl.append(p)
        predicted_labels = [-1 if x==0 else x for x in pl]
        print(i,total_te)
        print(predicted_labels.count(-1),predicted_labels.count(1))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))

    
train_NN()



-0.00017490705521919247
[0.16566417 0.18358776 0.22931115 0.00638735 0.26661503 0.2486865
 0.19304733]
[1.10181039 1.03044105 0.97258876 0.90323371 0.89967656 1.07621387
 0.98628538]
2199 615
0  d  (0.5766701050366574, 0.7089947089947091, 0.5835406301824212, None)


/home/vinay/snorkelEnv/lib/python3.5/site-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in double_scalars



nan
[       nan        nan 1.01434759 0.90080872        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
4000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)


/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
8000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
12000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
16000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
20000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)
0 nan
2814 0
(0.4664179104477612, 0.5, 0.4826254826254826, None)
(0.0, 0.0, 0.0, None)


/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
0  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
4000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
8000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
12000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[nan nan nan nan nan nan nan]
2814 0
16000  d  (0.4664179104477612, 0.5, 0.4826254826254826, None)

nan
[       nan        nan 1.01434759        nan        nan        nan
        nan]
[